In [1]:
# Se nombra los datos sobre los cuales vamos a trabajar
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1725723616900_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# SC = Spark Context
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

In [3]:
# Se usa para mostrar la base de datos df es la función. Spark.sql es para decir en qué lenguaje se va a hacer el query
df = spark.sql("Show databases") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Muestra las bases de datos consultadas anteriormente (estás son las de Glue).
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|    namespace|
+-------------+
| `db-taller1`|
|      default|
|   `labs- db`|
|myspectrum_db|
+-------------+

In [5]:
# Conociendo tipo de variables
spark.sql("SET hive.mapred.supports.subdirectories=true")
spark.sql("SET mapreduce.input.fileinputformat.input.dir.recursive=true")
 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[key: string, value: string]

In [6]:
# Aquí se llama el database a utilizar desde Glue
db_glue = spark.sql("use myspectrum_db") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Se llama la tabla del dataset
db_table = spark.sql("show tables in myspectrum_db") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Busca la tabla X del dataset Y 
db = spark.sql("select * from `db-taller1`.trabajo1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Muestra la tabla de arriba
db.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+---------+-------------+------------+--------------------+-----------+---------+---------+----+----+-----+----+-----------------+
|domain_code|              domain|area_code|         area|element_code|             element|months_code|   months|year_code|year|unit|value|flag| flag_description|
+-----------+--------------------+---------+-------------+------------+--------------------+-----------+---------+---------+----+----+-----+----+-----------------+
|Domain Code|              Domain|     null|         Area|        null|             Element|       null|   Months|     null|null|Unit| null|Flag| Flag Description|
|       "ET"|"Temperature change"|     null|"Afghanistan"|        null|"Temperature change"|       null|"January"|     null|null|"?C"| null|"Fc"|"Calculated data"|
|       "ET"|"Temperature change"|     null|"Afghanistan"|        null|"Temperature change"|       null|"January"|     null|null|"?C"| null|"Fc"|"Calculated data"|
|       "ET"|"Te

In [9]:
# Otra forma de cargar conociendo los formatos de las columnas

from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName("Cargar CSV desde S3").getOrCreate()

# Ruta del archivo S3
s3_path = "s3://sgonzalg2lab1/trabajo1/Trusted/dataset_taller1.csv"

# Cargar el archivo CSV desde S3 como un DataFrame de Spark
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(s3_path)

# Mostrar los primeros registros del DataFrame
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+---------------+-----------+------------+------------------+-----------+-------+---------+----+----+------+----+----------------+
|Domain Code|            Domain|Area Code (FAO)|       Area|Element Code|           Element|Months Code| Months|Year Code|Year|Unit| Value|Flag|Flag Description|
+-----------+------------------+---------------+-----------+------------+------------------+-----------+-------+---------+----+----+------+----+----------------+
|         ET|Temperature change|              2|Afghanistan|        7271|Temperature change|       7001|January|     1961|1961|  ?C| 0.746|  Fc| Calculated data|
|         ET|Temperature change|              2|Afghanistan|        7271|Temperature change|       7001|January|     1962|1962|  ?C| 0.009|  Fc| Calculated data|
|         ET|Temperature change|              2|Afghanistan|        7271|Temperature change|       7001|January|     1963|1963|  ?C| 2.695|  Fc| Calculated data|
|         ET|Temperature cha

In [10]:
# A partir de aquí, se puede hacer EDA
from pyspark.sql import functions as F

# Calcular el promedio de la columna 'Value'
promedio = df.agg(F.avg("Value").alias("Promedio")).collect()[0]["Promedio"]

# Mostrar el resultado
print(f"El promedio de la columna 'Value' es: {promedio}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

El promedio de la columna 'Value' es: 0.4926255833018091

In [20]:
df.describe().show(n=20, truncate=True, vertical=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+------------------+------------------+-----------+------------+------------------+-----------------+---------+------------------+------------------+------+------------------+------+------------------+
|summary|Domain Code|            Domain|   Area Code (FAO)|       Area|Element Code|           Element|      Months Code|   Months|         Year Code|              Year|  Unit|             Value|  Flag|  Flag Description|
+-------+-----------+------------------+------------------+-----------+------------+------------------+-----------------+---------+------------------+------------------+------+------------------+------+------------------+
|  count|     229925|            229925|            229925|     229925|      229925|            229925|           229925|   229925|            229925|            229925|229925|            222012|229925|            229925|
|   mean|       null|              null|130.64768946395563|       null|      7271.0|              null|7009.8823